# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Importing all the Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
file="D:/Homework/python-api-challenge/WeatherPy/Output_data/Cities.csv"

city_df = pd.read_csv(file)
city_df.head()



,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Morro Bay,35.3658,-120.8499,45.00,92,25,3.24,US,1616073534
1,Kamiiso,41.8167,140.6500,34.47,80,2,9.64,JP,1616073535
2,Busselton,-33.6500,115.3333,70.00,81,100,1.01,AU,1616073536
3,Port Elizabeth,-33.9180,25.5701,77.00,78,0,19.57,ZA,1616073537
4,Touros,-5.1989,-35.4608,82.40,78,75,8.05,BR,1616073538


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
#COnfiguring gmaps
gmaps.configure(api_key=g_key)

# Using the Lat and Lng as locations and Humidity as the weight
locations= city_df[["Lat","Lng"]]
weights=city_df["Humidity"]

#Setting the max_intensity to the maximum humidity value
maxhumidity= city_df["Humidity"].max()


#PLotting the heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

# Creating and adding the heat layer on the map 
heat_layer = gmaps.heatmap_layer(locations, weights= weights, 
                                 dissipating=False, max_intensity=maxhumidity,point_radius=1)

fig.add_layer(heat_layer)

# Displaing figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
#Creating an ideal dataframe with the cities with idal temperature, wind speed, humidity
ideal_df = city_df.loc[(city_df["Max Temp"]<80) & (city_df["Max Temp"]>70) &
                      (city_df["Cloudiness"]==0) &
                      (city_df["Wind Speed"] <=10) , :]


#Cleaning the ideal_df for better visualization
ideal_df= ideal_df.dropna(how='any')
ideal_df.reset_index(inplace =True)
del ideal_df["index"]

#Displaying the ideal dataframe
ideal_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Tezu,27.9167,96.1667,77.09,25,0,4.61,IN,1616073626
1,San Pedro de Ycuamandiyú,-24.0917,-57.0764,76.08,62,0,5.73,PY,1616073680
2,Cao Bang,22.6667,106.2500,74.10,87,0,4.65,VN,1616073619
3,Camotán,14.8167,-89.3667,71.60,73,0,1.63,GT,1616073785
4,Buritizeiro,-17.3511,-44.9622,79.86,64,0,4.63,BR,1616073823
5,Guarapuava,-25.3953,-51.4581,75.36,56,0,2.01,BR,1616073830
6,Iquique,-20.2208,-70.1431,73.40,56,0,8.05,CL,1616073584
7,Puerto El Triunfo,13.2833,-88.5500,75.20,88,0,3.78,SV,1616073877
8,Pimentel,-6.8367,-79.9342,73.99,78,0,3.44,PE,1616073888
9,Pāsighāt,28.0667,95.3333,77.02,34,0,4.07,IN,1616073995


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
#Creating a hotel dataframe with an new column "Hotel Name"
hotel_df= ideal_df.loc[:,["City","Country","Lat","Lng"]]
hotel_df["Hotel Name"] =""

#Displaying the Hotel dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Tezu,IN,27.9167,96.1667,
1,San Pedro de Ycuamandiyú,PY,-24.0917,-57.0764,
2,Cao Bang,VN,22.6667,106.2500,
3,Camotán,GT,14.8167,-89.3667,
4,Buritizeiro,BR,-17.3511,-44.9622,
5,Guarapuava,BR,-25.3953,-51.4581,
6,Iquique,CL,-20.2208,-70.1431,
7,Puerto El Triunfo,SV,13.2833,-88.5500,
8,Pimentel,PE,-6.8367,-79.9342,
9,Pāsighāt,IN,28.0667,95.3333,


In [12]:

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
        "keyword": "hotel",
        "radius": 5000,
        "type": "hotel",
        "key": g_key
            }

for index, row in hotel_df.iterrows():
    # getting  city name, lat, lnt from  hotel_df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assembling url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extracting results
    results = response['results']
    
    # saving the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, showing missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Waiting for 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Printing end of search once searching is completed
print("-------End of Search-------")


Retrieving Results for Index 0: Tezu.
Closest hotel in Tezu is Hotel Taboka.
------------
Retrieving Results for Index 1: San Pedro de Ycuamandiyú.
Closest hotel in San Pedro de Ycuamandiyú is DAUMINHA Hotel.
------------
Retrieving Results for Index 2: Cao Bang.
Closest hotel in Cao Bang is Max Boutique Hotel.
------------
Retrieving Results for Index 3: Camotán.
Missing field/result... skipping.
------------
Retrieving Results for Index 4: Buritizeiro.
Closest hotel in Buritizeiro is Hotel Pousada Sertão Veredas.
------------
Retrieving Results for Index 5: Guarapuava.
Closest hotel in Guarapuava is Hotel Slim Guarapuava.
------------
Retrieving Results for Index 6: Iquique.
Closest hotel in Iquique is Hilton Garden Inn Iquique, Chile.
------------
Retrieving Results for Index 7: Puerto El Triunfo.
Missing field/result... skipping.
------------
Retrieving Results for Index 8: Pimentel.
Closest hotel in Pimentel is Hotel Puerto del Sol.
------------
Retrieving Results for Index 9: Pās

In [13]:
#Displaying the whole dataframe with the Hotel Name
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Tezu,IN,27.9167,96.1667,Hotel Taboka
1,San Pedro de Ycuamandiyú,PY,-24.0917,-57.0764,DAUMINHA Hotel
2,Cao Bang,VN,22.6667,106.2500,Max Boutique Hotel
3,Camotán,GT,14.8167,-89.3667,
4,Buritizeiro,BR,-17.3511,-44.9622,Hotel Pousada Sertão Veredas
5,Guarapuava,BR,-25.3953,-51.4581,Hotel Slim Guarapuava
6,Iquique,CL,-20.2208,-70.1431,"Hilton Garden Inn Iquique, Chile"
7,Puerto El Triunfo,SV,13.2833,-88.5500,
8,Pimentel,PE,-6.8367,-79.9342,Hotel Puerto del Sol
9,Pāsighāt,IN,28.0667,95.3333,OYO 44691 Hotel East


In [14]:
# NOTE: Do not change any of the code in this cell


# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Adding the  marker layer on top of the heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Displaying Map and Figure
fig

Figure(layout=FigureLayout(height='420px'))